In [1]:
import os
import cv2
import numpy as np
from pathlib import Path
import subprocess
import shutil
import sys

In [2]:
# ROI Selection class
class ROISelector:
    def __init__(self):
        self.roi = None
        self.drawing = False
        self.start_point = None
        self.image = None
        self.clone = None
    
    def select_roi(self, video_path):
        """Let user select ROI from first frame of video"""
        cap = cv2.VideoCapture(str(video_path))
        ret, frame = cap.read()
        cap.release()
        
        if not ret:
            print(f"Error: Cannot read video {video_path}")
            return None
        
        self.image = frame.copy()
        self.clone = frame.copy()
        
        cv2.namedWindow("Select ROI - Press ENTER when done, 'r' to reset")
        cv2.setMouseCallback("Select ROI - Press ENTER when done, 'r' to reset", self.mouse_callback)
        
        print("\nDraw a rectangle around the object to remove:")
        print("- Click and drag to draw rectangle")
        print("- Press 'r' to reset")
        print("- Press ENTER when done")
        print("- Press ESC to skip this video\n")
        
        while True:
            cv2.imshow("Select ROI - Press ENTER when done, 'r' to reset", self.image)
            key = cv2.waitKey(1) & 0xFF
            
            if key == 13:  # Enter
                if self.roi is not None:
                    break
            elif key == ord('r'):  # Reset
                self.image = self.clone.copy()
                self.roi = None
            elif key == 27:  # ESC
                cv2.destroyAllWindows()
                return None
        
        cv2.destroyAllWindows()
        return self.roi
    
    def mouse_callback(self, event, x, y, flags, param):
        """Handle mouse events for ROI selection"""
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.start_point = (x, y)
        
        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing:
                self.image = self.clone.copy()
                cv2.rectangle(self.image, self.start_point, (x, y), (0, 255, 0), 2)
        
        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False
            cv2.rectangle(self.image, self.start_point, (x, y), (0, 255, 0), 2)
            x1, y1 = self.start_point
            x2, y2 = x, y
            self.roi = (min(x1, x2), min(y1, y2), abs(x2 - x1), abs(y2 - y1))

In [3]:
def create_mask_video(video_path, roi, output_mask_dir):
    """Create mask frames from ROI"""
    cap = cv2.VideoCapture(str(video_path))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    mask_dir = output_mask_dir / f"{video_path.stem}_mask"
    mask_dir.mkdir(exist_ok=True)
    
    x, y, w, h = roi
    
    print(f"Creating mask frames: {frame_count} frames...")
    frame_idx = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        mask = np.zeros((height, width), dtype=np.uint8)
        mask[y:y+h, x:x+w] = 255
        
        mask_filename = mask_dir / f"{frame_idx:05d}.png"
        cv2.imwrite(str(mask_filename), mask)
        frame_idx += 1
        
        if frame_idx % 100 == 0:
            print(f"  Processed {frame_idx}/{frame_count} frames")
    
    cap.release()
    
    print(f"Mask frames saved: {mask_dir}")
    return mask_dir

def run_opencv_inpaint(video_path, roi, output_video_path):
    """Fallback: Simple OpenCV inpainting (faster, lower quality)"""
    print(f"\nUsing OpenCV inpainting (fast method)...")
    
    cap = cv2.VideoCapture(str(video_path))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_video_path), fourcc, fps, (width, height))
    
    x, y, w, h = roi
    mask = np.zeros((height, width), dtype=np.uint8)
    mask[y:y+h, x:x+w] = 255
    
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        inpainted = cv2.inpaint(frame, mask, 3, cv2.INPAINT_TELEA)
        out.write(inpainted)
        
        frame_idx += 1
        if frame_idx % 100 == 0:
            print(f"  Processed {frame_idx}/{frame_count} frames")
    
    cap.release()
    out.release()
    
    print(f"✓ OpenCV inpainting completed")
    return True

def run_propainter(video_path, mask_path, output_video_path, propainter_dir):
    """Run ProPainter inference (disabled - use OpenCV fallback)"""
    print(f"\n⚠️ ProPainter has memory/performance issues with this video.")
    print(f"Skipping ProPainter...")
    return False

def process_video_with_propainter(video_path, output_dir, temp_dir, propainter_dir, use_opencv=True):
    """Process single video: ROI selection -> mask creation -> inpainting"""
    print(f"\n{'='*60}")
    print(f"Processing: {video_path.name}")
    print(f"{'='*60}")
    
    roi_selector = ROISelector()
    roi = roi_selector.select_roi(video_path)
    
    if roi is None:
        print("Skipped by user")
        return False
    
    print(f"ROI selected: x={roi[0]}, y={roi[1]}, w={roi[2]}, h={roi[3]}")
    
    output_video_path = output_dir / video_path.name
    
    if use_opencv:
        success = run_opencv_inpaint(video_path, roi, output_video_path)
    else:
        mask_path = create_mask_video(video_path, roi, temp_dir)
        success = run_propainter(video_path, mask_path, output_video_path, propainter_dir)
        try:
            shutil.rmtree(mask_path, ignore_errors=True)
        except:
            pass
    
    return success

In [4]:
def process_directory_tree(input_dir, output_dir, propainter_dir, use_opencv=True):
    """Process all videos in directory tree"""
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    propainter_dir = Path(propainter_dir)
    
    if not input_dir.exists():
        print(f"Error: Input directory does not exist: {input_dir}")
        return
    
    output_dir.mkdir(parents=True, exist_ok=True)
    
    temp_dir = output_dir / "_temp_masks"
    temp_dir.mkdir(exist_ok=True)
    
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv']
    video_files = []
    
    for ext in video_extensions:
        video_files.extend(input_dir.rglob(f'*{ext}'))
    
    if not video_files:
        print(f"No video files found in {input_dir}")
        return
    
    method = "OpenCV (fast)" if use_opencv else "ProPainter (slow, high quality)"
    print(f"\nFound {len(video_files)} video files to process")
    print(f"Input directory: {input_dir}")
    print(f"Output directory: {output_dir}")
    print(f"Method: {method}\n")
    
    processed = 0
    skipped = 0
    failed = 0
    
    for idx, video_path in enumerate(video_files, 1):
        print(f"\n[{idx}/{len(video_files)}] Processing video...")
        
        rel_path = video_path.relative_to(input_dir)
        output_video_dir = output_dir / rel_path.parent
        output_video_dir.mkdir(parents=True, exist_ok=True)
        
        output_video_path = output_video_dir / video_path.name
        if output_video_path.exists():
            print(f"Already exists: {output_video_path}")
            user_choice = input("Overwrite? (y/n): ").lower()
            if user_choice != 'y':
                skipped += 1
                continue
        
        success = process_video_with_propainter(video_path, output_video_dir, temp_dir, propainter_dir, use_opencv)
        
        if success:
            processed += 1
        else:
            failed += 1
    
    try:
        shutil.rmtree(temp_dir)
    except:
        pass
    
    print(f"\n{'='*60}")
    print(f"SUMMARY")
    print(f"{'='*60}")
    print(f"Total videos: {len(video_files)}")
    print(f"Processed: {processed}")
    print(f"Skipped: {skipped}")
    print(f"Failed: {failed}")
    print(f"{'='*60}")

In [ ]:
# Configuration
INPUT_DIR = r"C:\Users\thaim\Videos\AI_LEDS\second_dataset\detector_video"
OUTPUT_DIR = r"C:\Users\thaim\Videos\AI_LEDS\second_dataset\No_detector_video"
PROPAINTER_DIR = r"./ProPainter"

# Use OpenCV (fast, simpler) instead of ProPainter (slow, better quality)
USE_OPENCV = True

# Run processing
process_directory_tree(INPUT_DIR, OUTPUT_DIR, PROPAINTER_DIR, use_opencv=USE_OPENCV)


Found 55 video files to process
Input directory: C:\Users\thaim\Videos\AI_LEDS\DETECTOR
Output directory: C:\Users\thaim\Videos\AI_LEDS\NO_detector
Method: OpenCV (fast)


[1/55] Processing video...

Processing: 005_30_rgyl_ayn_shmsh_ldym_mkdymh_rka_shkvf.mp4

Draw a rectangle around the object to remove:
- Click and drag to draw rectangle
- Press 'r' to reset
- Press ENTER when done
- Press ESC to skip this video

ROI selected: x=182, y=421, w=76, h=90

Using OpenCV inpainting (fast method)...
  Processed 100/1917 frames
  Processed 200/1917 frames
  Processed 300/1917 frames
  Processed 400/1917 frames
  Processed 500/1917 frames
  Processed 600/1917 frames
  Processed 700/1917 frames
  Processed 800/1917 frames
  Processed 900/1917 frames
  Processed 1000/1917 frames
  Processed 1100/1917 frames
  Processed 1200/1917 frames
  Processed 1300/1917 frames
  Processed 1400/1917 frames
  Processed 1500/1917 frames
  Processed 1600/1917 frames
  Processed 1700/1917 frames
  Processed 180